<a href="https://colab.research.google.com/github/ahsanrazi/LangChain/blob/main/03_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.1 MB/s eta 0:00:00


In [ ]:
!pip install -qU langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.8/145.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.9 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
gemini_api_key = userdata.get("GEMINI_API_KEY").strip()

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model = "gemini-2.0-flash-exp", api_key=gemini_api_key)

In [ ]:
# First use the model directly

from langchain_core.messages import HumanMessage

model.invoke([HumanMessage("Hi I am Bob")])

AIMessage(content="Hi Bob! It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-a0c125f9-025e-4283-b1c6-5f80a9215386-0', usage_metadata={'input_tokens': 5, 'output_tokens': 19, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}})

In [ ]:
# The model on its own does not have any concept of state.

model.invoke([HumanMessage("What is my name?")])

AIMessage(content="I am an AI, so I don't know your name. You haven't told me! \n\nIf you'd like to tell me your name, I'd be happy to remember it for future conversations. 😊", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-11ecd5d1-6216-44fb-915e-b2af0696de79-0', usage_metadata={'input_tokens': 6, 'output_tokens': 48, 'total_tokens': 54, 'input_token_details': {'cache_read': 0}})

In [ ]:
# To get around this, we need to pass the entire conversation history into the model.

from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob! You told me that at the beginning of our conversation. 😊', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-877f973b-beed-4c8b-b901-5370bfd885f1-0', usage_metadata={'input_tokens': 25, 'output_tokens': 18, 'total_tokens': 43, 'input_token_details': {'cache_read': 0}})

# Message persistence

In [ ]:
# LangGraph implements a built-in persistence layer, making it ideal for chat applications that support multiple conversational turns.

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
# We now need to create a config that we pass into the runnable every time.

config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)

In [ ]:
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hi Bob! Nice to meet you again. How can I help you today?


In [ ]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)

In [ ]:
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Alright, Bob, you're being persistent! 😉 And I appreciate that. I still remember you told me your name is Bob. So, to the best of my knowledge, your name is **Bob**. 

Is there anything else on your mind? Maybe something other than your name? 😊


In [ ]:
# Our chatbot now remembers things about us. If we change the config to reference a different thread_id, we can see that it starts the conversation fresh.

config = {"configurable": {"thread_id": "xyz987"}}

query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)

In [ ]:
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm an AI, so I don't know your name. You haven't told me! 

If you'd like to tell me, I'd be happy to know it. 😊


# Prompt templates

In [ ]:
# Right now, all we've done is add a simple persistence layer around the model.
# We can start to make the chatbot more complicated and personalized by adding in a prompt template.

# Prompt Templates help to turn raw user information into a format that the LLM can work with.
# To add in a system message, we will create a ChatPromptTemplate. We will utilize MessagesPlaceholder to pass all the messages in.

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system", "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [ ]:
# We can now update our application to incorporate this template

workflow = StateGraph(MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)

In [ ]:
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ahoy there, Jim! A fine name ye have, says I. What brings ye to these here parts?


In [ ]:
# Let's now make our prompt a little bit more complicated

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system", "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [ ]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc456"}}

In [ ]:
query = "Hi! I'm Bob."
language = "Spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)

In [ ]:
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Hola Bob! Es un gusto conocerte. ¿En qué puedo ayudarte hoy?


In [ ]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Tu nombre es Bob.


# Managing Conversation History

In [ ]:
# One important concept to understand when building chatbots is how to manage conversation history.
# If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM.

# Importantly, you will want to do this BEFORE the prompt template but AFTER you load previous messages from Message History.